In [1]:
import pandas as pd
from datetime import datetime, timedelta
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score

In [2]:
df = pd.read_csv("flo_data_20k.csv")

In [3]:
print("### Veri Setine İlk Bakış ###")
df.head()

### Veri Setine İlk Bakış ###


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [4]:
print("\n### Veri Seti Bilgisi (Sütunlar, Tipler, Non-Null Değerler) ###")
df.info()


### Veri Seti Bilgisi (Sütunlar, Tipler, Non-Null Değerler) ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  fl

In [5]:
print("\n### Betimsel İstatistikler ###")
df.describe().T


### Betimsel İstatistikler ###


,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,149.98,286.46,578.44,45220.13


In [6]:
print(f"\n### Benzersiz Müşteri Sayısı: {df['master_id'].nunique()} ###")



### Benzersiz Müşteri Sayısı: 19945 ###


In [7]:
# Toplam alışveriş değerlerini tek bir sütunda toplama
df["customer_value_total"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]


In [8]:
print("\n### Toplam Değer ve Sipariş Sayısına Göre İlk 5 Müşteri ###")
df.sort_values(by="customer_value_total", ascending=False).head()



### Toplam Değer ve Sipariş Sayısına Göre İlk 5 Müşteri ###


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,customer_value_total,order_num_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0,2.0,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",45905.10,202.0
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.0,1.0,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",36818.29,68.0
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0,1.0,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",33918.10,82.0
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.0,1.0,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",31227.41,11.0
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.0,3.0,18119.14,2587.20,[AKTIFSPOR],20706.34,4.0


In [9]:
df.sort_values(by="order_num_total", ascending=False).head()


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,customer_value_total,order_num_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0,2.0,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",45905.10,202.0
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0,1.0,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",12282.24,131.0
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.0,109.0,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",10383.44,111.0
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.0,87.0,8432.25,139.98,[ERKEK],8572.23,88.0
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.0,81.0,3997.55,242.81,[ERKEK],4240.36,83.0


In [10]:
# Tarih sütunlarının tipini datetime'a çevirme
date_cols = [col for col in df.columns if "date" in col]

In [11]:
for col in date_cols:
    df[col] = pd.to_datetime(df[col])
print("\n### Tarih Sütunları Tipleri Güncellendi ###")
df[date_cols].dtypes


### Tarih Sütunları Tipleri Güncellendi ###


first_order_date           datetime64[ns]
last_order_date            datetime64[ns]
last_order_date_online     datetime64[ns]
last_order_date_offline    datetime64[ns]
dtype: object

In [12]:
# Analiz tarihi (Son sipariş tarihinden 2 gün sonrası)
max_date = df["last_order_date"].max()
today_date = max_date + timedelta(days=2)
print(f"\n### Analiz Tarihi (Today_Date): {today_date} ###")


### Analiz Tarihi (Today_Date): 2021-06-01 00:00:00 ###


In [13]:
# RFM metriklerinin hesaplanması
rfm = df.groupby('master_id').agg({
    "last_order_date": lambda last_order_date: (today_date - last_order_date.max()).days,
    "order_num_total": "sum",
    "customer_value_total": "sum"
})

In [14]:
# Sütun adlarını RFM formatına dönüştürme
rfm.columns = ['Recency', 'Frequency', 'Monetary']

In [15]:
print("\n### RFM Tablosu İlk 5 Gözlem ###")
rfm.head()


### RFM Tablosu İlk 5 Gözlem ###


,Recency,Frequency,Monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.0,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.0,269.47
000be838-85df-11ea-a90b-000d3a38a36f,213,4.0,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.0,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.0,1620.33


In [16]:
print("\n### RFM Metriklerinin Betimsel İstatistikleri ###")
rfm.describe().T


### RFM Metriklerinin Betimsel İstatistikleri ###


,count,mean,std,min,25%,50%,75%,max
Recency,19945.0,134.458360,103.281149,2.00,43.00,111.00,202.00,367.0
Frequency,19945.0,5.024768,4.742707,2.00,3.00,4.00,6.00,202.0
Monetary,19945.0,751.244287,895.402173,44.98,339.98,545.27,897.78,45905.1


In [17]:
# RF ve M skorlarının (1-5 arası) hesaplanması
# Recency için küçük değerler iyi olduğu için tersine çeviriyoruz (qcut'ta 5 en iyi skor)
rfm["R_Score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["F_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["M_Score"] = pd.qcut(rfm['Monetary'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])


In [18]:
# RFM Segmentlerinin oluşturulması
rfm["RF_SCORE"] = (rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str))


In [19]:
# Segment isimlerinin tanımlanması
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Lose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [20]:
# Segmentleri atama
rfm['Segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)


In [21]:
print("\n### RFM Segmentleri İlk 5 Gözlem ###")
rfm.head()


### RFM Segmentleri İlk 5 Gözlem ###


,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RF_SCORE,Segment
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.0,776.07,5,4,4,54,Champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.0,269.47,1,2,1,12,Hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213,4.0,722.69,2,3,4,23,At_Risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.0,874.16,5,4,4,54,Champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.0,1620.33,5,4,5,54,Champions


In [22]:
print("\n### RFM Segmentlerine Göre Metrik Ortalamaları ve Sayıları ###")
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])



### RFM Segmentlerine Göre Metrik Ortalamaları ve Sayıları ###


Recency        Frequency           Monetary      
                           mean count       mean count         mean count
Segment                                                                  
About_to_Sleep       114.031649  1643   2.406573  1643   361.649373  1643
At_Risk              242.328997  3152   4.470178  3152   648.325038  3152
Cant_Lose            235.159129  1194  10.716918  1194  1481.652446  1194
Champions             17.142187  1920   8.965104  1920  1410.708938  1920
Hibernating          247.426303  3589   2.391474  3589   362.583299  3589
Loyal_Customers       82.557926  3375   8.356444  3375  1216.257224  3375
Need_Attention       113.037221   806   3.739454   806   553.436638   806
New_Customers         17.976226   673   2.000000   673   344.049495   673
Potential_Loyalists   36.869744  2925   3.310769  2925   533.741344  2925
Promising             58.694611   668   2.000000   668   334.153338   668

In [23]:
# Gerekli CLTV metriklerini (Recency, T, Frequency, Monetary) hazırlama
# Recency_cltv: İlk ve son sipariş arasındaki geçen gün (müşteri yaşı)
# T: İlk sipariş ile analiz tarihi arasındaki geçen gün (toplam müşteri yaşı)
# Monetary_cltv: Müşteri başına ortalama kazanç (alışveriş başına ortalama)

# T metriklerinin hesaplanması
cltv_df = df.groupby('master_id').agg({
    'first_order_date': 'min',
    'last_order_date': 'max',
    'order_num_total': 'sum',
    'customer_value_total': 'sum'
})

In [24]:
cltv_df["Recency_cltv"] = (cltv_df["last_order_date"] - cltv_df["first_order_date"]).dt.days
cltv_df["T"] = (today_date - cltv_df["first_order_date"]).dt.days
cltv_df["Frequency"] = cltv_df["order_num_total"]
cltv_df["Monetary"] = cltv_df["customer_value_total"] / cltv_df["Frequency"] # Alışveriş başına ortalama kazanç


In [25]:
# BG/NBD Modeli
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['Frequency'],
        cltv_df['Recency_cltv'],
        cltv_df['T'])


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 500.99, b: 0.26, r: 3.48>

In [26]:
# 3 ve 6 aylık periyotlar için beklenen satın alma sayısını tahmin etme
cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                           cltv_df['Frequency'],
                                           cltv_df['Recency_cltv'],
                                           cltv_df['T'])
cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                           cltv_df['Frequency'],
                                           cltv_df['Recency_cltv'],
                                           cltv_df['T'])

In [27]:
print("\n### BG/NBD 3 ve 6 Aylık Tahminler İlk 5 Gözlem ###")
cltv_df.sort_values("exp_sales_6_month", ascending=False).head()



### BG/NBD 3 ve 6 Aylık Tahminler İlk 5 Gözlem ###


,first_order_date,last_order_date,order_num_total,customer_value_total,Recency_cltv,T,Frequency,Monetary,exp_sales_3_month,exp_sales_6_month
master_id,,,,,,,,,,
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,2020-02-16,2021-04-30,70.0,18443.57,439,471,70.0,263.479571,0.907139,1.814277
329968c6-a0e2-11e9-a2fc-000d3a38a36f,2019-02-14,2021-04-05,83.0,4240.36,781,838,83.0,51.088675,0.774999,1.549998
5d1c466a-9cfd-11e9-9897-000d3a38a36f,2013-10-11,2021-04-30,202.0,45905.10,2758,2790,202.0,227.252970,0.749237,1.498474
44d032ee-a0d4-11e9-a2fc-000d3a38a36f,2019-02-11,2021-02-11,77.0,5184.05,731,841,77.0,67.325325,0.719308,1.438616
03f502d4-a559-11e9-a2fc-000d3a38a36f,2019-02-21,2021-02-23,61.0,2758.47,733,831,61.0,45.220820,0.580840,1.161679


In [28]:
# Gamma-Gamma Modeli
# Monetary > 0 olanları filtreliyoruz (ilk alışverişten sonra tekrar alışveriş yapanlar)
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df[cltv_df["Frequency"] > 1]['Frequency'],
        cltv_df[cltv_df["Frequency"] > 1]['Monetary'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [29]:
# Beklenen ortalama karı tahmin etme
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(
    cltv_df['Frequency'],
    cltv_df['Monetary']
)

In [30]:
print("\n### Gamma-Gamma Tahminleri (Beklenen Ortalama Kar) İlk 5 Gözlem ###")
cltv_df.sort_values("exp_average_value", ascending=False).head()



### Gamma-Gamma Tahminleri (Beklenen Ortalama Kar) İlk 5 Gözlem ###


,first_order_date,last_order_date,order_num_total,customer_value_total,Recency_cltv,T,Frequency,Monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value
master_id,,,,,,,,,,,
47a642fe-975b-11eb-8c2a-000d3a38a36f,2021-04-07,2021-04-27,4.0,20706.34,20,55,4.0,5176.585000,0.161393,0.322785,5348.278535
7137a5c0-7aad-11ea-8f20-000d3a38a36f,2021-03-01,2021-04-13,11.0,31227.41,43,92,11.0,2838.855455,0.292977,0.585953,2872.560329
f59053e2-a503-11e9-a2fc-000d3a38a36f,2019-06-25,2020-06-21,7.0,7745.27,362,707,7.0,1106.467143,0.104066,0.208132,1127.612061
9083981a-f59e-11e9-841e-000d3a38a36f,2019-10-23,2021-01-10,4.0,4361.44,445,587,4.0,1090.360000,0.082474,0.164949,1127.355416
851de3b4-8f0c-11eb-8cb8-000d3a38a36f,2021-03-27,2021-05-24,2.0,1725.38,58,66,2.0,862.690000,0.115932,0.231863,923.681549


In [31]:
# CLTV Tahmini
cltv_pred_period = 6 # 6 aylık CLTV tahmini
cltv_df["CLTV"] = ggf.customer_lifetime_value(
    bgf,
    cltv_df['Frequency'],
    cltv_df['Recency_cltv'],
    cltv_df['T'],
    cltv_df['Monetary'],
    time=cltv_pred_period,  # Tahmin süresi (ay cinsinden)
    freq="W",  # T'nin frekansı (Haftalık) - Lifetimes kütüphanesine göre bu parametre ayarlanmalı
    discount_rate=0.01
)

In [32]:
print(f"\n### {cltv_pred_period} Aylık CLTV Tahminleri İlk 5 Gözlem ###")
cltv_df.sort_values("CLTV", ascending=False).head()


### 6 Aylık CLTV Tahminleri İlk 5 Gözlem ###


,first_order_date,last_order_date,order_num_total,customer_value_total,Recency_cltv,T,Frequency,Monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,CLTV
master_id,,,,,,,,,,,,
47a642fe-975b-11eb-8c2a-000d3a38a36f,2021-04-07,2021-04-27,4.0,20706.34,20,55,4.0,5176.585000,0.161393,0.322785,5348.278535,1811.318434
7137a5c0-7aad-11ea-8f20-000d3a38a36f,2021-03-01,2021-04-13,11.0,31227.41,43,92,11.0,2838.855455,0.292977,0.585953,2872.560329,1766.036618
d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,2018-08-06,2021-02-23,68.0,36818.29,932,1030,68.0,541.445441,0.560215,1.120431,542.523810,637.780883
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,2020-02-16,2021-04-30,70.0,18443.57,439,471,70.0,263.479571,0.907139,1.814277,264.019264,502.581879
5d1c466a-9cfd-11e9-9897-000d3a38a36f,2013-10-11,2021-04-30,202.0,45905.10,2758,2790,202.0,227.252970,0.749237,1.498474,227.416863,357.552270


In [33]:
# Recency, Frequency, Monetary zaten mevcut. Yeni özellikler ekleyelim:

# 1. Yaş (T - İlk siparişten bu yana geçen gün)
# cltv_df'den gelen T ve Frequency kullanışlı.

# 2. Channel bazlı metrikler (Online/Offline Oranları)
df_merged = df.merge(cltv_df[['T', 'Frequency', 'CLTV']], on='master_id', how='left')


In [34]:
# Offline/Online Sipariş Oranı
df_merged['Order_Ratio_Online'] = df_merged['order_num_total_ever_online'] / df_merged['order_num_total']
df_merged['Value_Ratio_Online'] = df_merged['customer_value_total_ever_online'] / df_merged['customer_value_total']


In [35]:
# 3. Kategori Sayısı
df_merged['Category_Count'] = df_merged['interested_in_categories_12'].apply(lambda x: len(x.replace('[', '').replace(']', '').split(',')) if x else 0)


In [36]:
# 4. Son sipariş kanalı (Kategorik)
df_merged = pd.get_dummies(df_merged, columns=['last_order_channel'], drop_first=True)


In [37]:
# 5. Churn Etiketi (Target Değişkeni)
# Son 3 ayda (90 gün) sipariş vermeyen müşteriyi Churn (1) olarak etiketleyelim.
churn_threshold_days = 90


In [38]:
# Son sipariş tarihinden Recency değerini alıyoruz (rfm tablosundan)
df_merged = df_merged.merge(rfm[['Recency']], on='master_id', how='left')


In [39]:
df_merged['Churn'] = df_merged['Recency'].apply(lambda x: 1 if x > churn_threshold_days else 0)


In [40]:
print(f"\n### Churn Dağılımı (>{churn_threshold_days} gün sipariş yok): ###")
df_merged['Churn'].value_counts(normalize=True)



### Churn Dağılımı (>90 gün sipariş yok): ###


Churn
1    0.572023
0    0.427977
Name: proportion, dtype: float64

In [41]:
# Kullanılacak özellikler (Recency, Frequency, T, Monetary, Online/Offline Oranları vb.)
features = [
    'Frequency', 'T', 'CLTV', 'order_num_total_ever_online',
    'order_num_total_ever_offline', 'customer_value_total_ever_online',
    'customer_value_total_ever_offline', 'Order_Ratio_Online',
    'Value_Ratio_Online', 'Category_Count'
]


In [42]:
# Churn değişkenini hedef alıyoruz
X = df_merged[features].fillna(0) # Yeni özellik seti
y = df_merged['Churn'] # Hedef değişken aynı


In [43]:
# Eğitim ve Test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

In [44]:
# Random Forest Modelini Eğitme
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
rf_model.fit(X_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [45]:
# Tahmin yapma
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

In [46]:
# Model Performansı
print("\n### Düzeltilmiş Model Performansı (Recency Çıkarıldı) ###")
print(f"Accuracy (Doğruluk): {accuracy_score(y_test, y_pred):.4f}")
print("--- Sınıflandırma Raporu ---")
print(classification_report(y_test, y_pred))


### Düzeltilmiş Model Performansı (Recency Çıkarıldı) ###
Accuracy (Doğruluk): 0.6153
--- Sınıflandırma Raporu ---
              precision    recall  f1-score   support

           0       0.61      0.28      0.39      2561
           1       0.62      0.86      0.72      3423

    accuracy                           0.62      5984
   macro avg       0.61      0.57      0.55      5984
weighted avg       0.61      0.62      0.58      5984



In [47]:
# ----- K-Fold Çapraz Doğrulama (Overfitting Kontrolü İçin) -----
cv = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = cross_val_score(rf_model, X, y, cv=cv, scoring='accuracy')

In [48]:
print("\n### 5-Fold Cross-Validation Accuracy Skorları ###")
print(f"Skorlar: {accuracy_scores}")
print(f"Ortalama CV Accuracy: {accuracy_scores.mean():.4f}")


### 5-Fold Cross-Validation Accuracy Skorları ###
Skorlar: [0.6242166  0.63098521 0.61093006 0.61368764 0.62471797]
Ortalama CV Accuracy: 0.6209


In [49]:
# Özellik Önem Sıralaması
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\n### Özellik Önem Sıralaması ###")
feature_importances


### Özellik Önem Sıralaması ###


CLTV                                 0.299318
T                                    0.276610
customer_value_total_ever_online     0.126775
Frequency                            0.093470
customer_value_total_ever_offline    0.051059
order_num_total_ever_online          0.042519
Category_Count                       0.031872
Value_Ratio_Online                   0.029944
Order_Ratio_Online                   0.029547
order_num_total_ever_offline         0.018887
dtype: float64

In [50]:
# 1. Test setindeki tahminleri ve olasılıkları bir DataFrame'e dönüştürelim
# master_id'yi geri almak için X_test'i index ile birleştiriyoruz.

# X_test, df_merged'den ayrıldığı için index'ini kullanarak master_id'yi geri alalım:
test_customers = df_merged.loc[X_test.index, ['master_id']]

In [51]:
# Test setindeki müşterilerin olasılıklarını ekleme
test_customers['Churn_Olasiligi'] = y_proba
test_customers['Tahmin_Sinifi'] = y_pred # (0 veya 1)

In [52]:
# Churn olasılığına göre sıralayarak en riskli müşterileri görme
print("\n### En Yüksek Churn Olasılığına Sahip Müşteriler (Test Seti) ###")
test_customers.sort_values(by="Churn_Olasiligi", ascending=False).head(10)


### En Yüksek Churn Olasılığına Sahip Müşteriler (Test Seti) ###


,master_id,Churn_Olasiligi,Tahmin_Sinifi
18594,c672f11a-aef8-11e9-a2fc-000d3a38a36f,0.770090,1
10157,0b2730fa-b002-11e9-9757-000d3a38a36f,0.762779,1
5522,ae164ea8-a9ff-11e9-a2fc-000d3a38a36f,0.762092,1
10181,2997ba62-a89d-11e9-a2fc-000d3a38a36f,0.761123,1
6560,4c3da95c-a6a7-11e9-a2fc-000d3a38a36f,0.756365,1
11232,f486e45e-a691-11e9-a2fc-000d3a38a36f,0.750997,1
2236,e4e7809a-603f-11ea-8dd8-000d3a38a36f,0.748903,1
3096,4bb3a040-acc9-11e9-a2fc-000d3a38a36f,0.747089,1
13609,da7ca3dc-9eb7-11e9-9897-000d3a38a36f,0.746036,1
12666,1af373d6-acbf-11e9-a2fc-000d3a38a36f,0.745490,1


In [53]:
# Yeni müşteri için varsayımsal değerler
# Bu değerler, yeni bir müşterinin (veya mevcut bir müşterinin) 
# verileri işlenerek elde edilmelidir.

yeni_musteri_ozellikleri = pd.DataFrame({
    # Müşterinin son siparişinden bu yana geçen gün sayısı (Recency)
    'Recency': [150], 
    
    # Müşterinin toplam sipariş sayısı (Frequency)
    'Frequency': [3], 
    
    # Müşterinin ilk siparişinden bu yana geçen gün sayısı (Müşteri Yaşı) (T)
    'T': [200], 
    
    # Müşteri Yaşam Boyu Değeri Tahmini (CLTV)
    'CLTV': [500.0], 
    
    # Online kanaldan verilen toplam sipariş sayısı
    'order_num_total_ever_online': [1.0], 
    
    # Offline kanaldan verilen toplam sipariş sayısı
    'order_num_total_ever_offline': [2.0], 
    
    # Online kanaldan elde edilen toplam parasal değer
    'customer_value_total_ever_online': [150.5], 
    
    # Offline kanaldan elde edilen toplam parasal değer
    'customer_value_total_ever_offline': [349.5], 
    
    # Online Sipariş Sayısı / Toplam Sipariş Sayısı Oranı
    'Order_Ratio_Online': [0.33],  # (1.0 / 3.0)
    
    # Online Parasal Değer / Toplam Parasal Değer Oranı
    'Value_Ratio_Online': [0.30],  # (150.5 / 500.0)
    
    # Müşterinin ilgilendiği kategori sayısı
    'Category_Count': [3.0] 
})

In [54]:
# Sütun sırasının modelin beklediği 'features' listesi ile aynı olduğundan emin olalım
yeni_musteri_ozellikleri = yeni_musteri_ozellikleri[features]

In [55]:
print("### Yeni Müşteri Özellikleri DataFrame'i ###")
yeni_musteri_ozellikleri.T

### Yeni Müşteri Özellikleri DataFrame'i ###


,0
Frequency,3.00
T,200.00
CLTV,500.00
order_num_total_ever_online,1.00
order_num_total_ever_offline,2.00
customer_value_total_ever_online,150.50
customer_value_total_ever_offline,349.50
Order_Ratio_Online,0.33
Value_Ratio_Online,0.30
Category_Count,3.00


In [56]:
# Churn olasılığını hesaplama
yeni_musteri_proba = rf_model.predict_proba(yeni_musteri_ozellikleri[features])[:, 1]

In [57]:
print(f"Churn Olasılığı: {yeni_musteri_proba[0]:.4f}")

Churn Olasılığı: 0.4721
